In [13]:
import requests
from bs4 import BeautifulSoup

In [21]:
urls=["https://www.ndtv.com/world-news/hamas-frees-2-more-hostages-amid-israels-300-strikes-in-24-hours-10-points-4508596"
, "https://sports.ndtv.com/icc-cricket-world-cup-2023/hurt-babar-azams-honest-admission-as-afghanistan-stun-pakistan-in-cricket-world-cup-2023-4508701#pfrom=home-ndtv_worldcup"]
responses=[]
for url in urls:
    response=requests.get(url)
    if response.status_code==200:
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        responses.append((text, response.url))
    else:
        print(f"Failed to retrieve the URL. Url={url} Status_code :{response.status_code}")

In [25]:
responses[1]

('"Hurt" Babar Azam\'s Honest Admission As Afghanistan Stun Pakistan In Cricket World Cup 2023 | Cricket Newsarrow-downredditsnapchattwitterfacebookcommentsemailprintersearchbelltop-navreddit-fillsnapchat-filltwitter-fillfacebook-fillsharewhatsapp-fillbookmarkNDTVWORLD EDITIONहिंदीWORLD CUP 2023 LiveMOVIESFOODLIFESTYLEHEALTHTECHBUSINESSGAMESBIG BONUSSWASTHSHOPPING SaleDon\'t MissFirst-class Niclas Fuellkrug Lands For Dortmund Ahead Of Bremen Reunion"Like Tiger In Cage": Roger Federer On Rafael Nadal\'s Game PersonalityAnushka Sharma\'s Four Word Reaction To Virat Kohli\'s Ton Against PakistanBangladesh\'s Mushfiqur Rahim To Miss India Match In Asia Cup 2023AdvertisementENG Select Languagesहिंदी | HindiWorld Cup 2023CricketFootballFormula 1TennisGolfBadmintonKabaddiNBABoxingAthleticsChessSwimmingShootingWrestlingTable tennisOthersportsLoginDark / Light mode"Hurt" Babar Azam\'s Honest Admission As Afghanistan Stun Pakistan In Cricket World Cup 2023facebooktwitterWhatsAppSettings Change F